In [2]:
import os 
import config as cfg
from dust3r.model import AsymmetricCroCo3DStereo 
import logging 
from dust3r.demo import get_reconstructed_scene, get_reconstructed_scene_with_known_poses
import matplotlib.pyplot as plt

logging.basicConfig(
    level=logging.INFO,  # Set the logging level to INFO
    format='%(asctime)s - %(levelname)s - %(message)s'  # Define the output format
)

device = cfg.DEVICE 
weights_path = cfg.MODEL_PATH   
model = AsymmetricCroCo3DStereo.from_pretrained(weights_path).to(device)
img_size = cfg.IMAGE_SIZE

# Log the device, and model path
logging.info(f"Device: {device}")
logging.info(f"Model Path: {weights_path}")

# Output directory 
output_dir = cfg.OUTPUT_DIR 
# Create the output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)


logging.info(f"Output Directory: {output_dir}")

# Data directory
data_dir = cfg.DATA_DIR
logging.info(f"Data Directory: {data_dir}")
filelist = os.listdir(data_dir) #return full path
filelist = [os.path.join(data_dir, x) for x in filelist]
logging.info(f"Filelist: {filelist}")


# Get the reconstructed scene
scene, outfile, imgs = get_reconstructed_scene_with_known_poses(
                            output_dir, model, device, cfg.SILENT, img_size, filelist, cfg.SCHEDULE,
                            cfg.NITER, cfg.MIN_CONF_THR, cfg.AS_POINTCLOUD, cfg.MASK_SKY,
                            cfg.CLEAN_DEPTH, cfg.TRANSPARENT_CAMS, cfg.CAM_SIZE, cfg.SCENEGRAPH_TYPE,
                            cfg.WINSIZE, cfg.REFID, cfg.KNOWN_POSES)

# Output is .glb file


... loading model from checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth
instantiating : AsymmetricCroCo3DStereo(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), landscape_only=False)
<All keys matched successfully>


2025-01-21 07:18:34,541 - INFO - Device: cuda
2025-01-21 07:18:34,541 - INFO - Model Path: checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth
2025-01-21 07:18:34,542 - INFO - Output Directory: temp_test/
2025-01-21 07:18:34,543 - INFO - Data Directory: /home/shashank/Documents/UniBonn/Sem5/aria-stereo-depth-completion/Priliminary tests/test_rectified_upright
2025-01-21 07:18:34,544 - INFO - Filelist: ['/home/shashank/Documents/UniBonn/Sem5/aria-stereo-depth-completion/Priliminary tests/test_rectified_upright/aria1_r_rectified_upright.jpeg', '/home/shashank/Documents/UniBonn/Sem5/aria-stereo-depth-completion/Priliminary tests/test_rectified_upright/aria1_l_rectified_upright.jpeg']


>> Loading a list of 2 images
 - adding /home/shashank/Documents/UniBonn/Sem5/aria-stereo-depth-completion/Priliminary tests/test_rectified_upright/aria1_r_rectified_upright.jpeg with resolution 512x512 --> 512x384
 - adding /home/shashank/Documents/UniBonn/Sem5/aria-stereo-depth-completion/Priliminary tests/test_rectified_upright/aria1_l_rectified_upright.jpeg with resolution 512x512 --> 512x384
 (Found 2 images)
>> Inference with model on 2 image pairs


100%|██████████| 2/2 [00:01<00:00,  1.58it/s]


mode GlobalAlignerMode.PairViewer
  - conf=7.46 for edge 0-1
  - conf=10.8 for edge 1-0


AttributeError: 'PairViewer' object has no attribute 'preset_pose'

In [ ]:

# print("images shape", len(imgs))
# print("First set of images shape", imgs[0].shape, imgs[1].shape, imgs[2].shape, imgs[3].shape, imgs[4].shape, imgs[5].shape)
# num_images = int(len(imgs)/3)
# for i in range(num_images):
#     ## Visualize the image, depth and confidence maps  at 3n-2, 3n-2, 3n indices
#     img = imgs[3*i]
#     depth = imgs[3*i+1]
#     confidence = imgs[3*i+2]
#     # Print max value in the depth map 
#     print("Max value in the depth map", depth.max())
#     print("Min value in the depth map", depth.min())    
#     print(f"Image {i}")
#     print(f"Image shape: {img.shape}")
#     print(f"Depth shape: {depth.shape}")
#     print(f"Confidence shape: {confidence.shape}")
#     print("\n")
#     # Visualize the image, depth and confidence maps in a single plot 
#     fig, ax = plt.subplots(1,3, figsize=(15,5))
#     ax[0].imshow(img)
#     ax[0].set_title("Image")
#     ax[1].imshow(depth)
#     ax[1].set_title("Depth")
#     ax[2].imshow(confidence)
#     ax[2].set_title("Confidence")
#     plt.show()




# # Visualize the scene

In [ ]:
print("Scene", scene.get_im_poses())
print("Scene", scene.get_im_poses().shape)
pose1 = scene.get_im_poses()[0] 
pose2 = scene.get_im_poses()[1]
# Find relative pose between two cameras 
def relative_pose(pose1, pose2):
    ''' 
    4*4 pose matrix of camera 1 and camera 2
    return the relative pose between camera 1 and camera 2 
    '''
    poseR1 = pose1[:3,:3]
    poseR2 = pose2[:3,:3]
    poseT1 = pose1[:3,3]
    poseT2 = pose2[:3,3]
    relative_pose = pose2
    relative_pose[:3,:3] = poseR1.T @ poseR2
    relative_pose[:3,3] = poseT2 - poseT1
    return relative_pose

relative_pose = relative_pose(pose1, pose2)
print("Relative Pose", relative_pose)

import numpy as np

def extract_translation_rotation(matrix):

    try:
        matrix = matrix.detach().cpu().numpy()  # Move to CPU and convert to NumPy array
    except:
        # If the matrix is already a NumPy array
        pass


    # Extract translation (last column)
    translation = matrix[:3, 3]
    translation = np.linalg.norm(translation)  # Compute the norm of the translation vector
    
    # Extract rotation (top-left 3x3 submatrix)
    rotation_matrix = matrix[:3, :3]
    
    # Convert rotation matrix to Euler angles (yaw, pitch, roll)
    yaw = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])  # Yaw (Z-axis)
    pitch = np.arcsin(-rotation_matrix[2, 0])  # Pitch (Y-axis)
    roll = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])  # Roll (X-axis)
    # Convert angles to degrees
    yaw = np.degrees(yaw)
    pitch = np.degrees(pitch)
    roll = np.degrees(roll)
    
    
    return translation, yaw, pitch, roll

# Extract translation and rotation from the relative pose
relative_translation, relative_yaw, relative_pitch, relative_roll = extract_translation_rotation(relative_pose)
print("Relative Translation", relative_translation)
print("Relative Yaw", relative_yaw)
print("Relative Pitch", relative_pitch)
print("Relative Roll", relative_roll)


# Test by transforming the pose1 to pose2
pose1_transformed = relative_pose @ pose1
print("Pose1 transformed", pose1_transformed)


In [ ]:
# gt_relative_pose_rgb_l = [[ 0.99339134 -0.05150564  0.10257099 -0.00428072]
#  [ 0.10364322  0.78649219 -0.60884162 -0.01184173]
#  [-0.04931251  0.61544878  0.78663275 -0.00511398]
#  [ 0.          0.          0.          1.        ]]

# gt_relative_pose_r_l = [[ 0.99836471 -0.03532317  0.04494634  0.00567676]
#  [ 0.05208901  0.23820291 -0.96981757 -0.11215309]
#  [ 0.02355068  0.97057285  0.23965332 -0.08683892]
#  [ 0.          0.          0.          1.        ]]
import numpy as np
gt_relative_pose_r_l = np.array([[ 0.99836471, -0.03532317,  0.04494634,  0.00567676],
    [ 0.05208901,  0.23820291, -0.96981757, -0.11215309],
    [ 0.02355068,  0.97057285,  0.23965332, -0.08683892],
    [ 0.,          0.,          0.,          1.        ]])

gt_relative_translation, gt_relative_yaw, gt_relative_pitch, gt_relative_roll = extract_translation_rotation(gt_relative_pose_r_l)
print("GT Relative Translation", gt_relative_translation)
print("GT Relative Yaw", gt_relative_yaw)
print("GT Relative Pitch", gt_relative_pitch)
print("GT Relative Roll", gt_relative_roll)


In [ ]:
known_poses = [[[1.0, 0.0, 0.0, 0.0],
               [0.0, 1.0, 0.0, 0.0],
               [0.0, 0.0, 1.0, 0.0],
               [0.0, 0.0, 0.0, 1.0]], 
                [[0.99339134, -0.05150564,  0.10257099, -0.00428072],
                [ 0.10364322,  0.78649219, -0.60884162, -0.01184173],
                [-0.04931251,  0.61544878,  0.78663275, -0.00511398],
                [ 0.0, 0.0, 0.0, 1.0]]]



In [ ]:
import open3d as o3d

# Load the .glb file
mesh = o3d.io.read_triangle_mesh("temp_test/scene.glb")


# Visualize the .glb file
o3d.visualization.draw_geometries([mesh])